# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content. 

In [ ]:
!python -m pip install --upgrade semantic-kernel
!python -m pip install --upgrade pip
!python -m pip install --upgrade tiktoken

In [1]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
import tiktoken
from semantic_kernel.events.function_invoking_event_args import FunctionInvokingEventArgs
from semantic_kernel.events.function_invoked_event_args import FunctionInvokedEventArgs



kernel = sk.Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat_gpt"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )


def calculate_input_tokens(any_string: str) -> int:
    # Create an instance of the tokenizer
    tokenizer = tiktoken.encoding_for_model("gpt-4")
    tokens = tokenizer.encode(any_string)
    return len(tokens)


def pre_invocation_handler(kernel_function_metadata, event_args: FunctionInvokingEventArgs):
    history = event_args.arguments.get('history', '')
    print(f"History: {history}")
    system_message = event_args.arguments.get('system_message', '')
    print(f"System Message: {system_message}")
    user_input = event_args.arguments.get('user_input', '')
    print(f"User Input: {user_input}")
    combined_text = f"{history} {system_message} {user_input}"
    print(f"Combined Text: {combined_text}")
    input_tokens = calculate_input_tokens(combined_text)
    
    # Log or send the input token count to your cost API
    print(f"Input Tokens: {input_tokens}")
    # Send input_tokens to your cost API


def post_invocation_handler(kernel_function_metadata, event_args: FunctionInvokedEventArgs):
    if event_args.function_result:
        # Extract token counts and model type
        token_counts = extract_token_counts_from_result(event_args.function_result)
        
        if token_counts:
            # Prepare and print the model type and token counts in a table format
            header = "| Token Type         | Count |"
            separator = "+--------------------+-------+"
            model_row = f"| Model Type         | {token_counts.pop('model_type', 'Unknown'):<5} |"
            
            print(separator)
            print(model_row)  # Print the model type row
            print(separator)
            print(header)
            print(separator)
            for token_type, count in token_counts.items():
                # Ensure we're not trying to print the model type again
                if token_type != 'model_type':
                    print(f"| {token_type:<18} | {count:>5} |")
            print(separator)
            
            # Here, you would send token_counts to your cost API
        else:
            print("Could not extract token counts from the function result.")


def extract_token_counts_from_result(function_result):
    # Ensure that the function_result object is not None
    if function_result is None:
        print("Function result is None.")
        return None
    
    # Access the inner content of the function result
    inner_content = function_result.get_inner_content()
    
    # Ensure inner_content is not None
    if inner_content is None:
        print("Inner content is None.")
        return None
    
    # Access the 'usage' attribute directly from the inner_content if it's structured as shown
    if hasattr(inner_content, 'usage'):
        usage_info = inner_content.usage
        
        # Extract token counts from the 'usage' attribute
        completion_tokens = getattr(usage_info, 'completion_tokens', 0)
        prompt_tokens = getattr(usage_info, 'prompt_tokens', 0)
        total_tokens = getattr(usage_info, 'total_tokens', 0)
        
        # Extract the model type from the inner_content
        model_type = getattr(inner_content, 'model', 'Unknown model')
        
        # Return the extracted token counts along with the model type
        return {
            'model_type': model_type,
            'completion_tokens': completion_tokens,
            'prompt_tokens': prompt_tokens,
            'total_tokens': total_tokens
        }
    else:
        print("Usage information is not available.")
        return None
    

# Assuming `kernel` is your instance of the semantic-kernel
# kernel.add_function_invoking_handler(pre_invocation_handler)
kernel.add_function_invoked_handler(post_invocation_handler)

Let's define a prompt outlining a dialogue chat bot.

In [3]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [4]:
if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.create_function_from_prompt(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [5]:
chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments

In [6]:
arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot

In [7]:
response = await kernel.invoke(chat_function, arguments)
print(response)

+--------------------+-------+
| Model Type         | gpt-4 |
+--------------------+-------+
| Token Type         | Count |
+--------------------+-------+
| completion_tokens  |    63 |
| prompt_tokens      |    76 |
| total_tokens       |   139 |
+--------------------+-------+
Of course, I'd be happy to help with that. Could you please provide a little more information? What type of books do you typically enjoy? For example, are you interested in fiction or non-fiction? Do you prefer a specific genre such as mystery, fantasy, romance, science fiction, history, etc?


Update the history with the output

In [8]:
chat_history.add_assistant_message(str(response))

Keep Chatting!

In [9]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    chat_history.add_user_message(input_text)

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_assistant_message(str(answer))

In [10]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
+--------------------+-------+
| Model Type         | gpt-4 |
+--------------------+-------+
| Token Type         | Count |
+--------------------+-------+
| completion_tokens  |   322 |
| prompt_tokens      |   176 |
| total_tokens       |   498 |
+--------------------+-------+
ChatBot: Certainly, based on your interest in history, philosophy, and Greece, I can suggest a few books:

1. "The Histories" by Herodotus: Known as 'The Father of History', Herodotus was a Greek historian who wrote a detailed record of the ancient world which includes a lot about ancient Greece.

2. "History of the Peloponnesian War" by Thucydides: This is a historical account of the Peloponnesian War (431–404 BC), which was fought between the Peloponnesian League (led by Sparta) and the Delian League (led by Athens). 

3. "Plato's Republic": Plato was one of the most important figures in Western philosophy. In "T

In [11]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
+--------------------+-------+
| Model Type         | gpt-4 |
+--------------------+-------+
| Token Type         | Count |
+--------------------+-------+
| completion_tokens  |    17 |
| prompt_tokens      |   505 |
| total_tokens       |   522 |
+--------------------+-------+
ChatBot: You asked about several books, which one specifically are you interested in knowing more about?


In [12]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
+--------------------+-------+
| Model Type         | gpt-4 |
+--------------------+-------+
| Token Type         | Count |
+--------------------+-------+
| completion_tokens  |    30 |
| prompt_tokens      |   551 |
| total_tokens       |   581 |
+--------------------+-------+
ChatBot: It seems like there's a bit of confusion. Could you please specify which book you're referring to? Then I can provide a more detailed explanation.


In [13]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
+--------------------+-------+
| Model Type         | gpt-4 |
+--------------------+-------+
| Token Type         | Count |
+--------------------+-------+
| completion_tokens  |   325 |
| prompt_tokens      |   600 |
| total_tokens       |   925 |
+--------------------+-------+
ChatBot: Absolutely, here are a few more recommendations for books about Greek history and philosophy:

1. "The World of Odysseus" by M.I. Finley: This book provides a fascinating look into the society and values of the Mycenaeans, the ancient Greeks of the Iliad and Odyssey.

2. "The Oresteia" by Aeschylus: This is a trilogy of Greek tragedies concerning the end of the curse on the House of Atreus. 

3. "The Symposium" by Plato: This is one of Plato's most famous works, in which various guests at a banquet each give a speech in honor of the god Eros and debate the nature of love. 

4. "The Nicomachean Ethics" by Aristotle: This is one of Aristo

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [14]:
print(chat_history)

<chat_history><message role="system">You are a helpful chatbot who is good about giving book recommendations.</message><message role="assistant">Of course, I'd be happy to help with that. Could you please provide a little more information? What type of books do you typically enjoy? For example, are you interested in fiction or non-fiction? Do you prefer a specific genre such as mystery, fantasy, romance, science fiction, history, etc?</message><message role="user">I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</message><message role="assistant">Certainly, based on your interest in history, philosophy, and Greece, I can suggest a few books:

1. "The Histories" by Herodotus: Known as 'The Father of History', Herodotus was a Greek historian who wrote a detailed record of the ancient world which includes a lot about ancient Greece.

2. "History of the Peloponnesian War" by Thucydides: This is a historical account of the Peloponnesian War (431–4